In [1]:
import PyPDF2 as ppdf
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import re
import os
import pandas as pd
import numpy as np
import math
import gspread
import panel as pn
from panel.interact import interact
pn.extension()
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.models import DatetimeTickFormatter, HoverTool

# [01]  Import PDFs
- User PyPDF2 to read texts in PDF file
- Clean and break components of texts to get labels (hierarchies) and numbers
- Append all pages together

In [ ]:
src = pd.DataFrame()
p01 = []

pdfobj = open('ข้อบัญญัติกรุงเทพฯ ว่าด้วยงบประมาณรายจ่ายประจำปี 2566.pdf', 'rb')
pdffile = ppdf.PdfReader(pdfobj)
npages = len(pdffile.pages[5:])

In [ ]:
begpage = 5

In [ ]:
def cleanpdf(pg):
    
    # Reading texts within pdf file
    df = pdffile.pages[pg].extract_text().strip()
    
    # Break itmes by the end of each numbers, followed by formatting
    df = df.replace('บาท\n','|').replace('\n',' ').replace('|','\n').replace('บาท','')
    df = df.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ')
    df = df.replace(',','')
    
    # After formatting, break text objects into lines, and feed into DataFrame
    df = df.splitlines()
    df = pd.DataFrame(df)
    
    # Take the last part to get only the numbers, and merge back to the DataFrame
    df = df.rename(columns={0: 'Raw'})
    df['Right'] = df['Raw'].str[-15:]
    df['Right2'] = df['Right'].str.split()
    df['Budgets'] = [n[-1] for n in df['Right2']]
#     df['Budgets'] = df['Right'].str.extractall('(\d+)').unstack()
#     df['Budgets'] = [s[e:] for (s, e) in zip(df['Right'], df['N'])]
    
    # Truncate only the texts (no numbers), as labels
    df['Raw2'] = [s.replace(r,'') for (s, r) in zip(df['Raw'], df['Budgets'])]
#     df['Raw2'] = df['Raw'].replace(df['Budgets'],'')
#     df['Raw2'] = [s[0:e] for (s, e) in zip(df['Raw'], df['L'])]
    
    # Rename columns
    df = df[['Raw2', 'Budgets']]
    
    # Assign flags for headers for hierarchies, as well as page numbers
    df['h1'] = df['Raw2'].str.contains(r'\(\d+\)').astype('int')
    df['h2'] = df['Raw2'].str.contains(r'^[กขค]\.').astype('int')
    df['flag page'] = df['Raw2'].str.contains(r'^\d+\ ').astype('int')
    df['T'] = np.where(df['flag page'] == 1, df['Raw2'].str.find(' '), 0)
    
    # Assign values based on flags
    df['Page'] = pd.to_numeric([s[0:e] for (s, e) in zip(df['Raw2'], df['T'])])
    df['Raw2'] = [s[e:] for (s, e) in zip(df['Raw2'], df['T'])]
    df['h1desc'] = np.where(df['h1'] == 1, df['Raw2'], np.nan)
    df['h2desc'] = np.where(df['h2'] == 1, df['Raw2'], np.nan)
    
    # Drop columns
    df = df.drop(columns={'T', 'flag page'})
    
    return df

In [ ]:
p01 = cleanpdf(begpage)
print(p01['Raw2'][0])
p01

In [ ]:
batch_from = 6
batch_to = 86

for i in np.arange(batch_from,batch_to):
    print(str("{:02}".format(i)), end=' ')
    ii = int(i)
    p02 = cleanpdf(ii)
    p01 = pd.concat([p01, p02])

In [ ]:
len(p01['Page'].drop_duplicates())

In [ ]:
p03 = p01
p03['Budgets'] = p03['Budgets'].str.extract('(\d+)').astype('int')
p03['Budgets Mn'] = (p03['Budgets'] / 1e6).astype('float64')
p03['Page'] = p03['Page'].fillna(method='ffill')
p03['h1desc'] = p03['h1desc'].fillna(method='ffill')
p03['h2desc'] = p03['h2desc'].fillna(method='ffill')
p03['h2desc'] = p03['h2desc'].fillna('')
p03.tail(10)

In [ ]:
p04 = p03[(p03['h1'] == 0) & (p03['h2'] == 0)]

# Removing leading blank spaces
p04['Raw2'] = np.where(p04['Raw2'].str.find(' ') == 0, p04['Raw2'].str[1:], p04['Raw2'])
p04['Raw2'] = np.where(p04['Raw2'].str.find('.') == 1, '0' + p04['Raw2'], p04['Raw2'])
p04['h1desc'] = np.where(p04['h1desc'].str.find(' ') == 0, p04['h1desc'].str[1:], p04['h1desc'])
p04['h1desc'] = np.where(p04['h1desc'].str.find(')') == 2, p04['h1desc'].str.replace('(','(0'), p04['h1desc'])

p04

In [ ]:
list_topics = [
      'Generic Administrations'
    , 'Maintenances'
    , 'Projects'
    , 'Communities'
    , 'Floods'
    , 'District Offices'
    , 'Human Resources'
    , 'Revenues'
    , 'Trainings'
    , 'Public Relations'
    , 'Welfares'
    , 'Waste Water Treatments'
    , 'Hygienic Foods'
    , 'Non-smoking Zones'
    , 'Cleanings'
    , 'Disease Controls'
    , 'Law Enforcements'
    , 'Green Zones'
    , 'Traffic Managements'
]    

list_keywords = [
      r'งานบริหารท'
    , r'รุงรักษา|งานดูแล|ปรับปรุง'
    , r'โครงการ'
    , r'พัฒนาชุมชน'
    , r'งานระบาย|ญหาน'
    , r'กงานเขต'
    , r'บุคลากร|บุคคล'
    , r'บรายได'
    , r'อบรม|หลักสูตร'
    , r'ประชาสัมพ'
    , r'สวัสดิการ'
    , r'บัดน(.*)เสีย'
    , r'อาหารปลอดภ'
    , r'เขตปลอดบุหรี่'
    , r'ความสะอาด'
    , r'โรค'
    , r'บังคับใช(.*)กฎหมาย'
    , r'สวน(.*)สีเขียว'
    , r'จราจร'
]

In [ ]:
p05 = pd.DataFrame()
p05 = p04

# Assign tags based on detected texts
def tags_keywords(lab,kw):
    p05[lab] = p05['Raw2'].str.contains(kw).astype('int')
    print(lab, kw, p05[p05[lab] == 1][lab].count())

for i in range(len(list_topics)):
    tags_keywords(list_topics[i], list_keywords[i])

In [ ]:
p06 = pd.DataFrame()
p06 = p05
p06['Chained Tags'] = 'All Categories'

chained_tags = p05.columns.to_list()[8:]
print(len(chained_tags))

for i in range(len(chained_tags)):
    tcol = 'tmpcol' + str(i)
    p06[tcol] = np.where(p06[chained_tags[i]] == 1, chained_tags[i], '')  
    p06['Chained Tags'] = p06['Chained Tags'] + '_' + p06[tcol]
    p06 = p06.drop(tcol, axis=1)
    
p06['Chained Tags'].drop_duplicates()

In [ ]:
list_topics2 = list_topics
list_topics2.append('All Categories')
print(list_topics2)

In [ ]:
print(p06.shape)

In [ ]:
listrank = [
      'h1desc'
    , 'h2desc'
    , 'Raw2'
]

numcol = ['Budgets Mn'] * len(listrank)

tmp = p06

for g , n in zip(listrank, numcol):
    tmp2 = tmp[[g,n]].groupby([g], as_index=False)[n].agg('sum')
    tmp2 = tmp2.sort_values(by=[n], ascending=False)
    rcol = 'Budget Ranks ' + g
    tmp2[rcol] = tmp2[n].rank(method='dense', ascending=False).astype('int')
    p06 = pd.merge(p06, tmp2[[g,rcol]], how='left', on=[g])

p06.head()

In [ ]:
disp = p06

# 

# [02] Define Widgets

- Filters 
 - Multiple drop-down lists
   - Departments
   - Impact levels (BAU/Strategic)
   - Tags (customised)
 - Text searches
 - Numeric budget hurdles as text box
 
 
- Sliders
 - Numeric budget hurdles
 - Ranks by Departments/Impact levels

#### [02-1] Define Keyword Tags for Multi-selections

In [ ]:
w_keywords = pn.widgets.MultiChoice(name='Categories by keywords (choose all that apply):', 
                                    value=['All Categories'], options=list_topics2)

pn.Column(w_keywords)

#### [02-2] Define slider for filterings of minimum budgets 

In [ ]:
w_slider_minbudgets = pn.widgets.IntSlider(start=0, end=5000, value=0, step=100, 
                                           name='Minimum Budget (THB millions)')

pn.Column(w_slider_minbudgets)

In [ ]:
w_slider_maxbudgets = pn.widgets.IntSlider(start=0, end=20000, value=20000, step=100, 
                                           name='Maximum Budget (THB millions)')

pn.Column(w_slider_maxbudgets)

#### [02-3] Define slider for filtering top ranks

In [ ]:
w_slider_topranks = pn.widgets.IntSlider(start=5, end=200, value=20, step=5, name='Top ranks')

pn.Column(w_slider_topranks)

#### [02-4] Define text box for containing substrings

In [ ]:
w_textsearch_h1 = pn.widgets.TextInput(name='Category (in Thai) contains:', value=' ')

pn.Column(w_textsearch_h1)

In [ ]:
w_textsearch_raw = pn.widgets.TextInput(name='Detailed descriptions (in Thai) contains:', value=' ')

pn.Column(w_textsearch_raw)

#### [02-5] Define switch button for h1desc and Raw2

In [ ]:
w_switch_hier = pn.widgets.RadioButtonGroup(value='Generic Categories', 
                                            options=['Generic Categories','Detailed Descriptions'])

pn.Column(w_switch_hier)

# 

# [03] Define charts receiving conditions from widgets

In [ ]:
numcol = 'Budgets Mn'
formatter = NumeralTickFormatter(format="0,0")

In [ ]:
@pn.depends(a=w_keywords 
            , b=w_slider_topranks
            , c=w_slider_minbudgets
            , d=w_slider_maxbudgets
            , e=w_textsearch_h1
           )
def f_keywordfilters_h1(a,b,c,d,e):
    chains = '|'.join(a)
    dfplot = disp[
                      (disp['Chained Tags'].str.contains(chains))
                    & (disp['h1desc'].str.contains(e))
                ]
    
    dfplot2 = dfplot[['h1desc',numcol]].groupby(['h1desc'], as_index=False)[numcol].agg('sum')
    dfplot2 = dfplot2.sort_values(by=[numcol]).tail(b)
    dfplot2 = dfplot2[
                      (dfplot2[numcol] >= c) 
                    & (dfplot2[numcol] <= d)
                ]
    hbar_h1 = dfplot2.hvplot.barh('h1desc', numcol, xlabel='', 
                                  width=800, height=600,
                                  ylabel='Budgets (THB millions)', line_color='white', xformatter=formatter, 
                                  title='{} Largest allocations of budgets'.format(b))
    return hbar_h1

pn.Column(
      pn.Row(
                w_keywords
              , w_textsearch_h1
              , w_textsearch_raw
      )
    , pn.Row(
                w_slider_topranks
              , w_slider_minbudgets
              , w_slider_maxbudgets
    )
    , pn.Row(f_keywordfilters_h1)
)

#### [02-5] Dashboard Templates

In [ ]:
template = pn.template.FastListTemplate(
    title='BMA Budget Allocations for Fiscal Year 2023',
    sidebar=[
        pn.pane.PNG('https://upload.wikimedia.org/wikipedia/commons/thumb/a/ad/Seal_Bangkok_Metropolitan_Admin_%28green%29.svg/1200px-Seal_Bangkok_Metropolitan_Admin_%28green%29.svg.png',
                   width=150, margin=(30,90))
        , pn.pane.Markdown('## Apply filters here to see only what you would like to see', margin=(20,10))
        , pn.Column(
                      w_keywords
                    , w_slider_topranks
                    , w_slider_minbudgets
                    , w_slider_maxbudgets
                    , w_textsearch_h1
                    )
    ]
    , main=[
          pn.Row(f_keywordfilters_h1)
    ]
    , theme_toggle=False
)

template.show()
# template.servable()